In [2]:
import os
import sys
import random
import numpy as np

from tqdm import tqdm
from rdkit.Chem import AllChem
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMolecule

from openff.units.elements import SYMBOLS
from openff.toolkit import Molecule
from openff.recharge.conformers import ConformerGenerator, ConformerSettings
from openff.recharge.esp import ESPSettings
from openff.recharge.esp.storage import MoleculeESPRecord, MoleculeESPStore
from openff.recharge.grids import LatticeGridSettings, GridSettingsType, GridGenerator
from openff.recharge.utilities.molecule import smiles_to_molecule
from qcelemental.models.procedures import OptimizationInput, QCInputSpecification
from openff.units import unit
from qcelemental.models.common_models import Model

import qcengine

sys.path.append('/Users/localadmin/Documents/projects/QM_ESP_Psi4')

from source.optimize.openff_psi4_gen import Psi4Generate
from source.conformers.conformer_gen import Conformers
from source.utilities.conversion_functions import conf_to_xyz_string


import psi4



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
#load our molecule to create the conformers

molecule = smiles_to_molecule("OCC(O)CO")
rdmol = molecule.to_rdkit()
AllChem.EmbedMultipleConfs(rdmol, numConfs=10, randomSeed=42)

: 

In [ ]:
#test the optimizer

optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
optimize_MMFF



: 

In [ ]:
#test all the conformers

for id in range(rdmol.GetNumConformers()):
    optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
    print(f'conformer:{id}, optimize (0 = converge, 1 = more iterations req): {optimize_MMFF}')

: 

In [ ]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import py3Dmol


: 

In [ ]:
#do the conformers look structurally sensible

for id in range(rdmol.GetNumConformers()):
    IPythonConsole.drawMol3D(rdmol,confId=id)


: 

In [ ]:
optimize_MMFF

: 

In [ ]:
#now create a series of xyz files so we can run the conformers with ASE, first convert our conformers to xyz format and then write to file

from openff.units import unit

cwd = os.getcwd()

conformers = []



for confs in range(rdmol.GetNumConformers()):
    conformer = np.zeros((rdmol.GetConformer(confs).GetNumAtoms(), 3))
    for atom_index, coordinates in enumerate(rdmol.GetConformer(confs).GetPositions()):
        conformer[atom_index, :] = coordinates
    conformers.append(conformer * unit.angstrom)


for number ,conformer in enumerate(conformers):
    atoms = [
            {
                "element": SYMBOLS[atom.atomic_number],
                "x": conformer[index, 0],
                "y": conformer[index, 1],
                "z": conformer[index, 2],
            }
            for index, atom in enumerate(molecule.atoms)
        ]
    xyz = f'{molecule.n_atoms}\n{molecule.to_smiles()}\n'
    for row in atoms:
        xyz += f"{row['element']}\t{np.around(row['x'].magnitude,decimals=6)}\t{np.around(row['y'].magnitude,decimals=6)}\t{np.around(row['z'].magnitude, decimals=6)}\n"
    try:
        f = open(cwd+f"/conformer_{number}.xyz", 'x')
        f.write(xyz)
        f.close()
    except FileExistsError:
        continue

: 

In [2]:


CWD = os.getcwd()

#generate water test molecule as openff.toolkit.Molecule
test_mol =  smiles_to_molecule('[H]O[H]')
conformer_list = Conformers.generate(test_mol, generation_type='rdkit')
conformer_list[0]
qc_mol =  test_mol.to_qcschema(conformer=0)

#Setup geometry optimisation
hf_model = Model(method="hf", basis="6-31G*")
spec = QCInputSpecification(model=hf_model, keywords={}, driver="gradient")
opt_spec = OptimizationInput(
            initial_molecule=qc_mol,
            input_specification=spec,
            keywords={"coordsys": "dlc", 
                      "program": "psi4"}                                        
        )

opt = qcengine.compute_procedure(opt_spec, "geometric")
print(f'geometry optimiztion was {opt.error}')
#return optimized molecule
optmized_mol = opt.final_molecule

#Generate grid.dat file for grid_esp and grid_field
grid_settings = LatticeGridSettings(
        type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
    )

grid = GridGenerator.generate(test_mol, optmized_mol.geometry*unit.angstrom, grid_settings)

grid = grid.to(unit.angstrom).m
np.savetxt("grid.dat", grid, delimiter=" ", fmt="%16.10f")
#compute one-electron properties.
opt_input_2 = { "molecule" : optmized_mol,
              "driver" : "energy",
              "model" : {"method":"scf","basis":"6-31G*"},
              "protocols":{"wavefunction":"all","stdout":True,"native_files":"all"},
              "keywords":{"scf_properties":["GRID_ESP", "GRID_FIELD","MULLIKEN_CHARGES", "LOWDIN_CHARGES", "DIPOLE", "QUADRUPOLE", "MBIS_CHARGES"]}                               
              }


opt_2 = qcengine.compute(opt_input_2, "psi4", task_config={"scratch_directory":CWD,"scratch_messy":True})

print(opt_2.dict())

esp = (np.loadtxt("grid_esp.dat").reshape(-1, 1) * unit.hartree / unit.e)

electric_field = (np.loadtxt("grid_field.dat")* unit.hartree/ (unit.e * unit.bohr))

rdkit has generated 10 conformers from a requested 10 conformers


NameError: name 'Model' is not defined

In [ ]:
psi4.core.variables(['MBIS Charges'])

: 

In [4]:
from source.optimize.openff_psi4_gen import Psi4Generate
import psi4
import qcengine

test_mol =  smiles_to_molecule('[H]O[H]')

#generate water test molecule as openff.toolkit.Molecule
test_mol =  smiles_to_molecule('[H]O[H]')

conformer_list = Conformers.generate(test_mol, generation_type='rdkit')
conformer_list[0]
qc_mol =  test_mol.to_qcschema(conformer=0)

#Generate grid.dat file for grid_esp and grid_field
grid_settings = LatticeGridSettings(
        type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
    )


esp_settings = ESPSettings(basis="6-31G*", method="hf", grid_settings=grid_settings)

xyz = Psi4Generate.run_calc(test_mol,conformer_list[0],esp_settings,compute_esp=True,compute_field=True, minimize=False)

molecule = psi4.geometry(
"""
3
[H]O[H]
H	-0.81024874	-0.18571238	-0.0
O	-0.00248133	0.36959931	-0.0
H	0.81273007	-0.18388693	0.0
"""
)

molecule.set_molecular_charge(0)
molecule.set_multiplicity(1)

E, wfn = psi4.energy('hf/6-31g*', return_wfn=True, molecule = molecule)
psi4.prop('hf/6-31G*', properties=["GRID_ESP", "GRID_FIELD","MULLIKEN_CHARGES", "LOWDIN_CHARGES", "DIPOLE", "QUADRUPOLE", "MBIS_CHARGES"])
#psi4.core.print_variables()
psi4.core.variables(['MBIS Charges'])
molecule.fcharge

rdkit has generated 10 conformers from a requested 10 conformers


AttributeError: type object 'Psi4Generate' has no attribute 'run_calc'

In [13]:
from source.optimize.openff_psi4_gen import Psi4Generate
from qcelemental.models.common_models import Model

test_mol =  smiles_to_molecule('[H]O[H]')

#generate water test molecule as openff.toolkit.Molecule
test_mol =  smiles_to_molecule('[H]O[H]')
conformer_list = Conformers.generate(test_mol, generation_type='rdkit')
conformer_list[0]
qc_mol =  test_mol.to_qcschema(conformer=0)



opt_spec = { "molecule" : qc_mol,
              "driver" : "gradient",
              "model" : {"method":"hf","basis":"6-31G*"}
            }

opt = qcengine.compute_procedure(opt_spec, "geometric", local_options = { "memory": 2000000, "ncores":4})
print(opt)
# print(opt.result)


rdkit has generated 10 conformers from a requested 10 conformers
FailedOperation(error=ComputeError(error_type='input_error', error_message="QCEngine Input Error: Error creating 'OptimizationInput', data could not be correctly parsed:\n5 validation errors for OptimizationInput\ninput_specification\n  field required (type=value_error.missing)\ninitial_molecule\n  field required (type=value_error.missing)\ndriver\n  extra fields not permitted (type=value_error.extra)\nmodel\n  extra fields not permitted (type=value_error.extra)\nmolecule\n  extra fields not permitted (type=value_error.extra)"))


/var/folders/5m/c625w21d6q3cgv41hc6yjs100000gq/T/ipykernel_18109/3519759898.py:19: FutureWarning: Using the `local_options` keyword argument is depreciated in favor of using `task_config`, in version 0.30.0 it will stop working.
  opt = qcengine.compute_procedure(opt_spec, "geometric", local_options = { "memory": 2000000, "ncores":4})


In [1]:
import os
import sys
import random
import numpy as np

from tqdm import tqdm
from rdkit.Chem import AllChem
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMolecule

from openff.units.elements import SYMBOLS
from openff.toolkit import Molecule
from openff.recharge.conformers import ConformerGenerator, ConformerSettings
from openff.recharge.esp import ESPSettings
from openff.recharge.esp.storage import MoleculeESPRecord, MoleculeESPStore
from openff.recharge.grids import LatticeGridSettings, GridSettingsType, GridGenerator
from openff.recharge.utilities.molecule import smiles_to_molecule
from qcelemental.models.procedures import OptimizationInput, QCInputSpecification
from openff.units import unit
from qcelemental.models.common_models import Model

sys.path.append('/Users/localadmin/Documents/projects/QM_ESP_Psi4')

from source.optimize.openff_psi4_gen import Psi4Generate
from source.conformers.conformer_gen import Conformers
from source.utilities.conversion_functions import conf_to_xyz_string


import psi4

sys.settrace 

test_mol =  smiles_to_molecule('[H]O[H]')

#generate water test molecule as openff.toolkit.Molecule
test_mol =  smiles_to_molecule('[H]O[H]')
conformer_list = Conformers.generate(test_mol, generation_type='rdkit')
conformer_list[0]
qc_mol =  test_mol.to_qcschema(conformer=0)

#Generate grid.dat file for grid_esp and grid_field
grid_settings = LatticeGridSettings(
        type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
    )
grid = GridGenerator.generate(test_mol, conformer_list[0], grid_settings)

grid = grid.to(unit.angstrom).m
np.savetxt("grid.dat", grid, delimiter=" ", fmt="%16.10f")


esp_settings = ESPSettings(basis="6-31G*", method="hf", grid_settings=grid_settings)

xyz = conformer_string = conf_to_xyz_string(conformer_list[0], test_mol)

psi4.set_output_file('output.dat')

molecule = psi4.core.Molecule.from_string("""
                                            nocom
                                            noreorient
                                            H	-0.81024874	-0.18571238	-0.0
                                            O	-0.00248133	0.36959931	-0.0
                                            H	0.81273007	-0.18388693	0.0 
                                          """)

molecule.set_molecular_charge(0)
molecule.set_multiplicity(1)

psi4.set_options({"DFT_SPHERICAL_POINTS":"590",
                  "DFT_RADIAL_POINTS":"99"})

E, wfn = psi4.prop('hf/6-31G*', properties=["GRID_ESP", 
                                            "GRID_FIELD",
                                            "MULLIKEN_CHARGES", 
                                            "LOWDIN_CHARGES", 
                                            "DIPOLE", 
                                            "QUADRUPOLE", 
                                            "MBIS_CHARGES"],
                                            molecule = molecule, 
                                            return_wfn = True,
                                            )  #"GRID_ESP", "GRID_FIELD"
psi4.core.variables(['MBIS Charges'])
wfn.variables()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
rdkit has generated 10 conformers from a requested 10 conformers


{'CURRENT ENERGY': -76.00688623049763,
 'CURRENT REFERENCE ENERGY': -76.00688623049763,
 'DD SOLVATION ENERGY': 0.0,
 'HF KINETIC ENERGY': 75.71208232169353,
 'HF POTENTIAL ENERGY': -151.71896855219114,
 'HF TOTAL ENERGY': -76.00688623049763,
 'HF VIRIAL RATIO': 2.0038937498449916,
 'NUCLEAR REPULSION ENERGY': 8.941189202753957,
 'ONE-ELECTRON ENERGY': -122.62733060211329,
 'PCM POLARIZATION ENERGY': 0.0,
 'PE ENERGY': 0.0,
 'SCF ITERATION ENERGY': -76.00688623049763,
 'SCF ITERATIONS': 8.0,
 'SCF TOTAL ENERGY': -76.00688623049763,
 'TWO-ELECTRON ENERGY': 37.679255168861694,
 'HF DIPOLE': array([ 0.00408617, -0.83522958,  0.        ]),
 'HF QUADRUPOLE': array([[-2.78962776e+00,  4.75167568e-03,  0.00000000e+00],
        [ 4.75167568e-03, -5.40692800e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -5.41213293e+00]]),
 'LOWDIN CHARGES': array([ 0.36108223, -0.72231233,  0.3612301 ]),
 'MBIS CHARGES': array([[ 0.46457588],
        [-0.92803681],
        [ 0.46346084]]),


In [ ]:
prop

: 

In [ ]:
wfn.variables()

: 

In [ ]:
molecule = psi4.core.Molecule.from_string("""
                                            nocom
                                            noreorient
                                            H	-0.81024874	-0.18571238	-0.0
                                            O	-0.00248133	0.36959931	-0.0
                                            H	0.81273007	-0.18388693	0.0 
                                          """)

molecule.to_string('xyz')


: 

In [ ]:
mulliken_charges = wfn.variable("MULLIKEN_CHARGES")
lowdin_charges = wfn.variable("LOWDIN_CHARGES")
dipole = wfn.variable("HF DIPOLE")
quadrupole = wfn.variable("HF QUADRUPOLE")
mbis_charges = wfn.variable("MBIS CHARGES")

: 

In [ ]:
variable_names = ["MULLIKEN_CHARGES", "LOWDIN_CHARGES", "HF DIPOLE", "HF QUADRUPOLE", "MBIS CHARGES"]
variables = {name: wfn.variable(name) for name in variable_names}

: 

In [ ]:
from qcelemental.models import Molecule as QCMol
qc_mol =  test_mol.to_qcschema(conformer=0)
qc_mol = QCMol(**qc_mol.dict(exclude={"fix_com", "fix_orientation"}), fix_com=True, fix_orientation=True)
molecule = psi4.geometry(qc_mol.to_string("psi4"))
qc_mol.geometry

: 

In [1]:
import sys
import os
CWD = os.getcwd()
sys.path.append(os.path.dirname(CWD))

from source.inputSetup.SmilesInputs import ReadInput
from source.optimize.esp_generator_wrapper import ESPGenerator, PropGenerator
from source.conformers.conformer_gen import Conformers
from openff.recharge.utilities.molecule import smiles_to_molecule
from openff.recharge.grids import LatticeGridSettings
from openff.recharge.esp import ESPSettings



    
#Read the .smi input and add to list
smiles = ReadInput.read_smiles('test_files.smi')

# Define the grid that the electrostatic properties will be trained on and the
# level of theory to compute the properties at.
grid_settings = LatticeGridSettings(
    type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
)

esp_settings = ESPSettings(basis="6-31G*", method="hf", grid_settings=grid_settings)

#Loop through molecules
for mol in smiles:
    molecule = smiles_to_molecule(mol)
    #Generate the conformers
    conformer_list = Conformers.generate(molecule, generation_type='rdkit')
    ESP_gen = PropGenerator(molecule = molecule, conformers = conformer_list, esp_settings = esp_settings, grid_settings = grid_settings)
    ESP_gen.memory = 2e+9 #2gb
    print(f'number of cores is {ESP_gen.ncores}')
    print(f'memory is {ESP_gen.memory}')
    ESP_gen.run_props()
    #ESP_gen.fetch_data()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
rdkit has generated 10 conformers from a requested 10 conformers


ERROR:tornado.general:SEND Error: Host unreachable


KeyboardInterrupt: 

In [6]:
import psi4
psi4.set_memory('2gb')


  Memory set to   1.863 GiB by Python driver.


2000000000

In [ ]:
import ipykernel

: 

: 

In [ ]:
!mamba install -c conda-forge xtb

: 